In [13]:
print("hello world")

hello world


In [14]:
# basics
import pandas as pd 
import numpy as np
import os 
import re
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()
import requests
import urllib

# plotting
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import plotly.express as px
import seaborn as sns

# modeling
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.tree import DecisionTreeRegressor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:
# local or gdrive
path_source = 'local2'

if path_source == 'gdrive':
  from google.colab import drive
  drive.mount('/content/gdrive')
  data_path = '/content/gdrive/MyDrive/Classes/W210_capstone/W210_Capstone/Data/joined_data'
  #env_path = '/content/gdrive/MyDrive/.env'
  
elif path_source == 'local':
  data_path = '/Users/tj/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone/W210_Capstone/Data/joined_data'
  #env_path = '/content/gdrive/MyDrive/.env'

elif path_source == 'local2':
  data_path = '/data/modeling_data'
  #env_path = '/content/gdrive/MyDrive/.env'

In [4]:
  
def current_path():
    print("Current working directory before")
    print(os.getcwd())
    print()
   
current_path()
   
os.chdir('../')
   
current_path()

Current working directory before
c:\Users\anandadmin\Documents\ML_MS\UCB\2022_Fall\W210\JLPS_capstone_project\modeling

Current working directory before
c:\Users\anandadmin\Documents\ML_MS\UCB\2022_Fall\W210\JLPS_capstone_project



In [5]:
os.getcwd()

'c:\\Users\\anandadmin\\Documents\\ML_MS\\UCB\\2022_Fall\\W210\\JLPS_capstone_project'

In [6]:
# load full dataset
df = pd.read_csv('data\modeling_data\modeling_data_2022-10-16.csv')

In [7]:
df.head()

,year_month,school_zip,school_county_v2,school_region_name,pm25,school_elevation_m,ps_elevation_m,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,total_pop_under19,pop_under19_male,pop_under19_female,total_population,total_population_male,total_population_female,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wspd_merge,school_wdir_wrt_0n,ps_wdir_wrt_0n,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,nearby_point_source_count,school_wspd,ca_agi_per_returns,total_tax_liability,tax_liability_per_capita,school_temperature,ps_temperature,school_count,pm25_last_month,pm25_r6,pm25_r9,pm25_r12,pm25_r24,pm25_slope6,pm25_slope9,pm25_slope12,pm25_slope24,pm25_lag_12mo,year,month,school_region_name_central_coast,school_region_name_inland_empire,school_region_name_los_angeles_county,school_region_name_north_coast,school_region_name_northern_san_joaquin_valley,school_region_name_orange_county,school_region_name_san_diego_-_imperial,school_region_name_san_francisco_bay_area,school_region_name_southern_san_joaquin_valley,school_region_name_superior_california,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,2000-01-01,90001,Los Angeles,Los Angeles County,32.149998,44.728889,43.703333,6196.0,3209.0,2987.0,6672.0,3397.0,3275.0,5562.0,2850.0,2712.0,5075.0,2599.0,2476.0,23505.0,12055.0,11450.0,54481.0,27320.0,27161.0,14.241154,3854.812685,-90.196586,0.757031,-172.758321,-172.758321,82.561735,82.561735,0.757031,82.561735,1.124995,0.000000,0.757031,20049.704556,2608176.0,47.873130,14.277778,14.266667,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2000-01-01,90002,Los Angeles,Los Angeles County,31.849998,33.858889,29.690000,4795.0,2447.0,2348.0,5655.0,2870.0,2785.0,5077.0,2566.0,2511.0,4316.0,2179.0,2137.0,19843.0,10062.0,9781.0,44584.0,21553.0,23031.0,6.649500,2734.278190,-75.730039,0.757031,-172.758321,-172.758321,97.028283,97.028283,0.757031,97.028283,0.879819,0.222222,0.757031,19697.001063,1549733.0,34.759847,14.300000,14.400000,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2000-01-01,90003,Los Angeles,Los Angeles County,31.724998,42.329167,45.785000,6527.0,3297.0,3230.0,6894.0,3539.0,3355.0,5994.0,3000.0,2994.0,5381.0,2727.0,2654.0,24796.0,12563.0,12233.0,58187.0,28557.0,29630.0,11.672797,5281.522927,-63.495073,0.855611,-172.758321,-60.057878,106.478652,100.850655,0.806321,103.664653,0.869845,0.000000,0.757031,18895.491452,2255523.0,38.763349,14.300000,14.383333,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2000-01-01,90004,Los Angeles,Los Angeles County,29.560000,83.190000,102.906667,5621.0,2879.0,2742.0,5505.0,2816.0,2689.0,4470.0,2297.0,2173.0,4204.0,2199.0,2005.0,19800.0,10191.0,9609.0,67850.0,34200.0,33650.0,5.158316,6452.615217,-87.100171,0.757031,-172.758321,-172.758321,78.897449,78.897449,0.757031,78.897449,1.160977,0.000000,0.757031,41990.032278,44046571.0,649.175696,14.022222,13.911111,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2000-01-01,90006,Los Angeles,Los Angeles County,30.750000,65.920000,77.570000,5939.0,3050.0,2889.0,6009.0,2995.0,3014.0,4493.0,2337.0,2156.0,4416.0,2370.0,2046.0,20857.0,10752.0,10105.0,62765.0,31901.0,30864.0,3.553121,5187.184865,-78.360123,0.757031,-172.758321,-172.758321,94.398199,94.398199,0.757031,94.398199,0.923498,0.000000,0.757031,17629.297337,3132557.0,49.909297,14.140000,14.100000,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [8]:
df.columns

Index(['year_month', 'school_zip', 'school_county_v2', 'school_region_name',
       'pm25', 'school_elevation_m', 'ps_elevation_m', 'population_0_4',
       'population_0_4_male', 'population_0_4_female', 'population_5_9',
       'population_5_9_male', 'population_5_9_female', 'population_10_14',
       'population_10_14_male', 'population_10_14_female', 'population_15_19',
       'population_15_19_male', 'population_15_19_female', 'total_pop_under19',
       'pop_under19_male', 'pop_under19_female', 'total_population',
       'total_population_male', 'total_population_female',
       'point_source_pm25_tpy', 'dist_school_to_ps_m', 'angle_to_school',
       'ps_wspd_merge', 'school_wdir_wrt_0n', 'ps_wdir_wrt_0n',
       'school_wind_alignment', 'ps_wind_alignment', 'avg_wind_speed',
       'avg_wind_alignment', 'avg_wind_alignment_cosine',
       'nearby_point_source_count', 'school_wspd', 'ca_agi_per_returns',
       'total_tax_liability', 'tax_liability_per_capita', 'school_temperatu

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312345 entries, 0 to 312344
Data columns (total 78 columns):
 #   Column                                          Non-Null Count   Dtype  
---  ------                                          --------------   -----  
 0   year_month                                      312345 non-null  object 
 1   school_zip                                      312345 non-null  int64  
 2   school_county_v2                                312345 non-null  object 
 3   school_region_name                              312345 non-null  object 
 4   pm25                                            312345 non-null  float64
 5   school_elevation_m                              312345 non-null  float64
 6   ps_elevation_m                                  312345 non-null  float64
 7   population_0_4                                  312345 non-null  float64
 8   population_0_4_male                             312345 non-null  float64
 9   population_0_4_female     

In [10]:
temp = df.isna().sum()

In [16]:
display(temp.sort_values(ascending=False))

pm25_r24                                          33312
pm25_slope24                                      33312
pm25_lag_12mo                                     16656
pm25_slope12                                      16656
pm25_r12                                          16656
pm25_r9                                           12492
pm25_slope9                                       12492
pm25_slope6                                        8328
pm25_r6                                            8328
pm25_last_month                                    1388
tax_liability_per_capita                             24
school_region_name_los_angeles_county                 0
school_count                                          0
ps_temperature                                        0
school_temperature                                    0
year                                                  0
month                                                 0
school_region_name_central_coast                